# Import libraries

In [1]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
import ot
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

In [3]:
#-- Set base directory and data directory path --#
basePath   = '/Users/bobli/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/'
# basePath   = '/Users/hanchengli/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/'
dataPath   = 'Data/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

# Functions

In [4]:
%cd ~/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions
%run centralFunctions.ipynb

/Users/bobli/Library/CloudStorage/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions


# Loading Data

In [5]:
# create a dictionary to store data
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

In [6]:
# store data in dictionart as numpy array
bkg_data = dataDict['bkg']['Particles'][:,:,0:3]
sig_data = {}

for alias in sigAliasList:
  sig_data[alias] = dataDict[alias]['Particles'][:,:,0:3]

# Low $p_T$ range

In [7]:
# set basic parameters
nEvents = 1000
random_state = Generator(PCG64(123))
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [8]:
total_event_pT = {}

# calculate total event pT for bkgs and signals
total_event_pT['bkg'] = np.sum(bkg_data[:, :, 0], axis=1)
for alias in sigAliasList:
    total_event_pT[alias] = np.sum(sig_data[alias][:,:,0], axis=1)


pTrange = [0,50,100,150,200,500,1000]
neighbor_list = list(range(5, 400,10))

avg_aucs = []
std_aucs = []
avg_ks = []
std_ks = []

# iterate through pT ranges and for each signal type do an kNN classification
scoreDict = {}
for i in range(0, len(pTrange)-1):
    lower_bound = pTrange[i]
    upper_bound = pTrange[i+1]
    scoreDict[str(pTrange[i+1])] = {}
    
    filtered_events = {}
    mask = (total_event_pT['bkg'] >= lower_bound) & (total_event_pT['bkg'] <= upper_bound)
    
    filtered_events['bkg'] = randomDataSample(bkg_data[mask],nEvents,random_state)
    np.random.seed(i)
    permutation = np.random.permutation(nEvents*2)
    
    for alias in sigAliasList:
        
        mask = (total_event_pT[alias] >= lower_bound) & (total_event_pT[alias] <= upper_bound)
        filtered_events[alias] = randomDataSample(sig_data[alias][mask],nEvents,random_state)
        
        event_list = np.concatenate((filtered_events['bkg'],filtered_events[alias]))
        event_labels = np.asarray([0] * nEvents + [1] * nEvents)
        event_list = event_list[permutation]
        event_labels = event_labels[permutation]
        
        distance_matrix = calcOTDistance(event_list, event_list, OTSCHEME, '2D', Matrix = True)
        
        avg_auc, std_auc, avg_k, std_k, metrics_dict = kNN_cross_validation(distance_matrix, event_labels, neighbor_list, k_fold=5)
        print(avg_auc, std_auc, avg_k, std_k)
        avg_aucs.append(avg_auc)
        std_aucs.append(std_auc)
        avg_ks.append(avg_k)
        std_ks.append(std_k)
        scoreDict[str(pTrange[i+1])][alias] = metrics_dict

Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 56.86it/s]


0.6193241561554373 0.013909554559565948 25.0 12.649110640673518


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 58.22it/s]


0.5560362022323223 0.026013491872497616 85.0 87.40709353364863


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 66.57it/s]


...

0.5829378691399614 0.021238964288079334 83.0 63.686733312362634


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 69.99it/s]


0.6202817253317441 0.0341775808408001 81.0 74.45804187594514


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 70.14it/s]


0.5284963612145445 0.015364811569643429 57.0 80.34923770640266


In [9]:
print(avg_aucs)
print(std_aucs)

[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287, 0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318, 0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865, 0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944, 0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541, 0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]
[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139, 0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385, 0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753, 0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636, 0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634, 0.02230500501928155, 0.021238964288079334, 0.0341775808408001, 0.015364811569643429]


In [10]:
grouped_data_1 = [avg_aucs[i:i+4] for i in range(0, len(avg_aucs), 4)]
grouped_data_2 = [std_aucs[i:i+4] for i in range(0, len(std_aucs), 4)]
print(grouped_data_1)
print(grouped_data_2)

[[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287], [0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318], [0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865], [0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944], [0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541], [0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]]
[[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139], [0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385], [0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753], [0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636], [0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634], [0.02230500501928155, 0.021238964288079334, 0.0341775808408001,

In [11]:
print(scoreDict)

{'50': {'sig_A': {'repeat0': (0.6196181252509032, array([0.        , 0.00471698, 0.00591195, 0.00886792, 0.01122642,
       0.0134434 , 0.01867925, 0.02533019, 0.03034591, 0.03404088,
       0.03773585, 0.04143082, 0.04512579, 0.04882075, 0.05377358,
       0.06485849, 0.07580713, 0.08368973, 0.09157233, 0.09945493,
       0.10733753, 0.11526587, 0.12332762, 0.13138937, 0.13945111,
       0.14751286, 0.15557461, 0.16396226, 0.17283019, 0.18169811,
       0.19056604, 0.19767296, 0.2039544 , 0.21023585, 0.2165173 ,
       0.22279874, 0.22908019, 0.23536164, 0.24164308, 0.24792453,
       0.25420597, 0.26048742, 0.26676887, 0.27305031, 0.2811186 ,
       0.2893531 , 0.2975876 , 0.3058221 , 0.3140566 , 0.32229111,
       0.33052561, 0.34097242, 0.35393324, 0.36689405, 0.37985486,
       0.39281567, 0.40577649, 0.4187373 , 0.42809605, 0.43454545,
       0.44099485, 0.44744425, 0.45389365, 0.46034305, 0.46679245,
       0.47324185, 0.47969125, 0.48614065, 0.49259005, 0.49903945,
       0.507

In [12]:
import json
%cd ~/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/distance_matrix_classification/kNN
for pT in scoreDict:
    dict = scoreDict[pT]
    converted_dict = {
        row: {
            col: [dict[row][col][0]] + [arr.tolist() for arr in dict[row][col][1:]]
            for col in dict[row]
        }
        for row in dict
    }
    
    with open('kNN_2D_planed_'+pT+'GeV.json', 'w') as json_file:
        json.dump(converted_dict, json_file, indent=4)

/Users/bobli/Library/CloudStorage/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/distance_matrix_classification/kNN


In [ ]:
transposed_dict = {
    col: {row: scoreDict[row][col] for row in scoreDict}
    for col in scoreDict[next(iter(scoreDict))]
}
# Convert NumPy arrays in the lists to regular lists
converted_dict = {
    row: {
        col: [transposed_dict[row][col][0]] + [arr.tolist() for arr in transposed_dict[row][col][1:]]
        for col in transposed_dict[row]
    }
    for row in transposed_dict
}

with open('kNN_distance_matrix_3D.json', 'w') as json_file:
    json.dump(converted_dict, json_file, indent=4)